In [1]:
# Colab environment already has nltk
!pip install nltk

In [2]:
# -*- coding: utf-8 -*-
"""
TA Workshop : Text preparation step by step
"""

import os
import json
import string
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
import pandas as pd

In [3]:
#load required data for NLTK 
## Note: on local machine, you just need to run it once only using "nltk.download()"
##       at the 'NLTK Downloader' dialogue window, choose 'book', then click 'Download'

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anirban\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anirban\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anirban\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# Open the text data file from your current working directory
data = open("osha.txt", "r")
docslist = data.readlines()

listRows = []
for row in docslist:
    listRows.append(row.split('\t'))

df = pd.DataFrame(listRows)
df.columns = ['Date','Title','Description']

print("There are %d records and %d columns"%(df.shape[0],df.shape[1]))

There are 10000 records and 3 columns


In [5]:
df['Title_Length'] = df['Title'].apply(lambda x: len(x.split())-1)
df.head(10)

,Date,Title,Description,Title_Length
0,201079928,Employee Is Burned By Forklift Radiator Fluid,At approximately 11:30 a.m. on November 13 2...,6
1,202561825,Employee Falls From Flatbed Trailer And Later...,On August 30 2013 Employee #1 was working f...,7
2,200361855,Two Workers Are Struck By Motor Vehicle And O...,On August 27 2013 Employees #1 and #2 of T...,10
3,200361863,Employee Is Struck By Bales Of Wire And Killed,On August 26 2013 Employee #1 with Lee Iro...,8
4,201079324,Employee Is Splashed With Hot Water And Is Bu...,On July 14 2013 Employee #1 vacuum pump tr...,8
5,202658258,Employee Suffers Burns While Moving Soup,On June 30 2013 Employee #1 was working in ...,5
6,202685947,Employee Injures Self With Knife,An incident occurred as Employee #1 was attem...,4
7,202673471,Foreman Is Fatally Crushed When Forklift Tips...,At approximately 6:30 a.m. on May 13 2013 E...,7
8,202369575,Employee Fractures Abdomen When Run Over By T...,On April 23 2013 Employee #1 a baggage tug...,8
9,202509832,Employee Suffers Abdominal Fracture In Fall F...,On April 9 2013 Employee #1 was installing ...,7


In [6]:
print('There are {} titles with single word description'.format(len(df[df['Title_Length']==1])))

There are 390 titles with single word description


In [7]:
df['number_of_tokens'] = df.Description.apply(lambda x: len(word_tokenize(x)))
df['number_of_tokens'].max()

804

In [8]:
df[df['number_of_tokens']==3]['Description'].unique()

array(['InspectionOpen DateSICEstablishment Name\n'], dtype=object)

In [9]:
import re
from nltk.stem import SnowballStemmer
 

class StemmerTokenizer(object):
    def _init_(self):
        self.stemmer = SnowballStemmer(language='english')
        self.stop = set(stopwords.words('english'))

 

    def _call_(self, doc):
        # Remove special characters and hybrid word-punctuation characters
        doc = re.sub(r"[^A-Za-z0-9]", " ", doc)

 

        # tokenize text into word tokens
        tokens = word_tokenize(doc)

 

        # strip out punctuation & integers
        words = [t for t in tokens if t.isalpha()]


        #convert to lowercase
        words = [t.lower() for t in words]
        
        # strip out stopwords
        words = [t for t in words if t not in self.stop]
        
        # lemmatize each token
        return [self.stemmer.stem(t) for t in words]